# Tema 3 - Cifrado de flujo con ChaCha20

Las prácticas de este curso las haremos en Python, conde tenemos dos posibilidades para probar criptografía: el paquete PyCryptodome y el paquete cryptography. Ambos son opciones válidas e intercambiables. Las prácticas de este curso las haremos con PyCryptodome. Puedes encontrar la ayuda en: https://pycryptodome.readthedocs.io/en/latest/

Si estás siguiendo estas notas en tu PC, puedes instalarlo con: `python3 -m pip install pycryptodome`. Si las estás siguiendo en Colab, ejecuta la siguiente línea:

In [1]:
!python3 -m pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00


A continuación importamos las librerías que vamos a necesitar.

In [2]:
from base64 import b64encode, b64decode
from Crypto.Cipher import ChaCha20
from Crypto.Random import get_random_bytes

## Configuración del sistema emisor

Los módulos de criptografía suelen necesitar una etapa inicial de configuración. Cada módulo se configura a su manera. A continuación encontrarás la etapa de configuración de ChaCha20 para PyCryptodome.

Fíjate que la clave se crea al azar con algoritmos criptográficos `Crypto.Random.get_random_bytes()`: **es fundamental que las claves sean totalmente aleatorias y creadas también con algoritmos criptográficos**. No todas las funciones de creación de azar son válidas: necesitas una fuente de azar con validez criptográfica. Muchas implementaciones de protocolos criptográficos han caído no porque el cifrado fuese erróneo, sino porque la fuente de azar no era lo suficiente "aleatoria". En particular: en criptografía, no uses la función general `random.randbytes()` sino alguna específica de la librería criptográfica, como `Crypto.Random.get_random_bytes()`. Esto es válido también para los demás sistemas criptográficos.

In [15]:
key = get_random_bytes(32)
cipher_emisor = ChaCha20.new(key=key, nonce=None)
print('Longitud de la clave: {} bits'.format(8 * len(key)))

Longitud de la clave: 256 bits


En PyCryptodome el *nonce* se puede pasar al algoritmo durante la configuración. Si, como en este caso, no se pasa *nonce* durante la creación, la librería crea un *nonc* al azar que podemos recuperar. Si decides crear tú el *nonce*, recuerda que también tiene que ser un número aleatorio creado con algoritmos criptográficos, igual que la clave.

In [16]:
nonce = cipher_emisor.nonce
print(f'nonce creado automáticamente: {b64encode(nonce)}, longitud: {8 * len(nonce)} bits')

# si lo necesitas:
# nonce_creado_por_mi = Crypto.Random.get_random_bytes(8)
# cipher_emisor = ChaCha20.new(key=key, nonce=once_creado_por_mi)

nonce creado automáticamente: b'pltFqWU3yhI=', longitud: 64 bits


## Cifrado del mensaje por el emisor

El emisor cifra el mensaje `Atacaremos al amanecer` y envía al receptor `result`, es decir, la pareja "mensaje cifrado" y "*nonce*". Fíjate: el nonce se puede enviar por un canal inseguro, así que se asume que el atacante lo conocerá.

Observa que el resultado lo codificamos en Base64 (https://es.wikipedia.org/wiki/Base64). Aunque no es necesario, sí que es común hacerlo así porque algunos protocolos (correo electrónico, JSON...) solo puede enviar caracteres imprimibles. No pierdes ni ganas seguridad si decides usar o no Base64, es más una exigencia de tu sistema de comunicaciones. Fíjate que he usado la expresión "codificamos en Base64", no ciframos. Base64 es un algoritmo de codificación de bytes, no tiene claves, cualquiera lo puede codificar y decodificar y por tanto Base64 no es un cifrado.

In [17]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
result = {'nonce':b64encode(nonce), 'ciphertext':b64encode(ciphertext)}
print(result)

{'nonce': b'pltFqWU3yhI=', 'ciphertext': b'97ua7gGzM/0K7SSVuHnnDSK/q1eSbA=='}


## Recepción y descifrado

El receptor toma el *nonce* y el *ciphertext*, primero decodifica el base64, configura el *cipher* con la clave secreta que tiene que conocer previamente, y después descifra.

En temas posteriores veremos cómo el receptor conoce la clave secreta.

In [18]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

b'Atacaremos al amanecer'


## Siguientes mensajes: sincronización entre ciphers

Supongamos que el usuario vuelve a enviar el mismo mensaje, con el mismo cipher (fíjate que no volvemos a definir `cipher_emisor`: lo estamos reaprovechando)

In [20]:
plaintext = b'Atacaremos al amanecer'
ciphertext = cipher_emisor.encrypt(plaintext)
result = {'nonce':b64encode(nonce), 'ciphertext':b64encode(ciphertext)}
print(result)

{'nonce': b'pltFqWU3yhI=', 'ciphertext': b'UIpTSjbYLaORBZyZmldQXF2/B3wAnA=='}


Fíjate: estamos cifrando el mismo mensaje con el mismo nonce... pero el ciphertext es diferente. ¿Recuerdas que nunca se debe cifrar el mismo texto con la misma clave? ChaCha20 nos ayuda a que no lo hagamos, ni siquiera por equivocación, mediante el uso de un contador.

Supongamos que el receptor crea un nuevo cipher, con la misma configuración de key y nonce, e intenta descifrar:

In [21]:
received_nonce = b64decode(result['nonce'])
received_ciphertext = b64decode(result['ciphertext'])
cipher_receptor = ChaCha20.new(key=key, nonce=received_nonce)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

b'\xe6E\xa8\xc7V\x19{3\xf4\x9b\x98mN\x0e\xd6<\x1en\xc9H\xf7\x82'


¿Qué ha pasado? ¿Por qué no se descifra? Recuerda que ChaCha20 tiene un contador adicional interno. Es decir: **emisor y receptor tienen que estar sincronizados** Es decir: para descifrar el byte número 22 tenemos que decirle al receptor que han pasado 22 bytes antes, aunque no los haya visto.

(nota: 22 es el tamaño en bytes de la cadena "Atacaremos al amanecer", que fue el contenido del primer mensaje)

Si volvemos a intentar descifrar, ahora sí que podemos hacerlo:

In [22]:
cipher_receptor.seek(22)
plaintext = cipher_receptor.decrypt(received_ciphertext)
print(plaintext)

b'\'G\x17\xc6|\x06\xb8\x9c\r=\xbd\xfa!\xa3\t"m\x02\x96\xc7R\x87'


PyCryptodome y todos los demás están sincronizados siempre que descifremos los mismos bytes que hemos cifrado desde que se han creado los dos ciphers, el de emisión y el de recepción.

Si alguno de los dos pierde la sincronización (por ejemplo, porque se reinicia), entonces es necesario volver a sincronizarlos con un "seek": "ya envié XX bytes aunque no los hayas visto, mueve el estado a esta posición"

Poder volver a sincronizar los dos streams es una enorme ventaja de ChaCha20 y eso es por el parámetro `pos` autoincremental que forma parte de la matriz de estado. No todos los algoritmos permiten sincronizar los flujos si se pierde la sincronización.

<font color="#f00"> (Este ejercicio no tiene preguntas)</font>